# Answer Questions with Chicago Crime Data

Shenyue Jia

## Imports

In [37]:
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format',lambda x: f"{x:,.2f}")

## Data preparation

- Chicago crime data
    - Source: [Chicago Data Portal](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2)
    - CSV cleaned and splitted by years (link)
- Holiday data
    - [Share URL](https://docs.google.com/spreadsheets/d/1d8hoZzDAhbWx6EwNjrMTTOE5-23Pr1VxJeUxVj1JL9U/edit?usp=sharing)

### Holiday data

In [38]:
# read holiday data
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTMR4vi1nFtbYO_WLqecU6WoRAQ0F-_QCMcuWUoZhuTygAP2U668cddCWBeAV2eop9fFdyXX856AB2j/pub?output=csv'
holiday = pd.read_csv(url)
holiday.head()

,Date,US Holidays,State Holidays
0,2001-01-01,New Year's Day,New Year's Day
1,2001-01-02,NaN,NaN
2,2001-01-03,NaN,NaN
3,2001-01-04,NaN,NaN
4,2001-01-05,NaN,NaN


### Read and Clean Chicago crime data

In [39]:
## (Required) MAKE SURE TO CHANGE THIS VARIABLE TO MATCH YOUR LOCAL FILE NAME
RAW_FILE = "/Users/Shenyue/Downloads/Crimes_-_2001_to_Present.csv" #(or slice correct index from the crime_files list)

if RAW_FILE != "/Users/Shenyue/Downloads/Crimes_-_2001_to_Present.csv":
	raise Exception("You must update the RAW_FILE variable to match your local filepath.")
	
RAW_FILE

'/Users/Shenyue/Downloads/Crimes_-_2001_to_Present.csv'

In [40]:
chicago_full = pd.read_csv(RAW_FILE)
chicago_full.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,924,9.00,12.00,61.00,08B,"1,165,074.00","1,875,917.00",2015,02/10/2018 03:50:01 PM,41.82,-87.67,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,1511,15.00,29.00,25.00,06,"1,138,875.00","1,904,869.00",2015,02/10/2018 03:50:01 PM,41.90,-87.77,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,631,6.00,8.00,44.00,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,1412,14.00,35.00,21.00,18,"1,152,037.00","1,920,384.00",2015,02/10/2018 03:50:01 PM,41.94,-87.72,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,1522,15.00,28.00,25.00,08A,"1,141,706.00","1,900,086.00",2015,02/10/2018 03:50:01 PM,41.88,-87.76,"(41.881903443, -87.755121152)"


In [41]:
# explicitly setting the format to speed up pd.to_datetime
date_format = "%m/%d/%Y %I:%M:%S %p"


### Demonstrating/testing date_format
example = chicago_full.loc[0,'Date']
display(example)
pd.to_datetime(example,format=date_format)

'09/05/2015 01:30:00 PM'

Timestamp('2015-09-05 13:30:00')

In [24]:
# this cell can take up to 1 min to run
chicago_full['Datetime'] = pd.to_datetime(chicago_full['Date'], format=date_format)
chicago_full = chicago_full.sort_values('Datetime')
chicago_full = chicago_full.set_index('Datetime')
chicago_full

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
Datetime,,,,,,,,,,,,,,,,,,,,,,
2001-01-01 01:00:00,6154338,HP242745,01/01/2001 01:00:00 PM,065XX S KEDZIE AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,831,8.00,15.00,66.00,06,"1,156,214.00","1,861,165.00",2001,04/02/2008 01:05:33 AM,41.77,-87.70,"(41.774818809, -87.702896431)"
2001-01-01 01:00:00,1311503,G001102,01/01/2001 01:00:00 AM,026XX N HALSTED ST,0264,CRIM SEXUAL ASSAULT,AGGRAVATED: OTHER DANG WEAPON,BAR OR TAVERN,False,False,1933,19.00,NaN,NaN,02,"1,170,510.00","1,918,326.00",2001,08/17/2015 03:03:40 PM,41.93,-87.65,"(41.931373805, -87.648818936)"
2001-01-01 01:00:00,1318099,G003019,01/01/2001 01:00:00 AM,041XX S PRAIRIE AV,0460,BATTERY,SIMPLE,RESIDENCE PORCH/HALLWAY,False,True,214,2.00,NaN,NaN,08B,"1,178,685.00","1,877,637.00",2001,08/17/2015 03:03:40 PM,41.82,-87.62,"(41.819537938, -87.62002027)"
2001-01-01 01:00:00,1311358,G001819,01/01/2001 01:00:00 PM,025XX W 45 PL,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,914,9.00,NaN,NaN,05,"1,160,341.00","1,874,463.00",2001,08/17/2015 03:03:40 PM,41.81,-87.69,"(41.811226273, -87.687401084)"
2001-01-01 01:00:00,1422085,G143929,01/01/2001 01:00:00 AM,016XX S CALIFORNIA AV,2820,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,True,1023,10.00,NaN,NaN,26,"1,158,003.00","1,891,509.00",2001,08/17/2015 03:03:40 PM,41.86,-87.70,"(41.858050399, -87.695512503)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-25 12:50:00,13021773,JG198585,03/25/2023 12:50:00 PM,040XX N HAMLIN AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,1723,17.00,45.00,16.00,14,"1,150,341.00","1,926,496.00",2023,04/01/2023 04:49:15 PM,41.95,-87.72,"(41.95421088, -87.722722794)"
2023-03-25 12:50:00,13021160,JG198614,03/25/2023 12:50:00 PM,021XX N MASON AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,2515,25.00,36.00,19.00,14,"1,136,391.00","1,913,416.00",2023,04/01/2023 04:49:15 PM,41.92,-87.77,"(41.918579241, -87.77431911)"
2023-03-25 12:50:00,13021465,JG198588,03/25/2023 12:50:00 PM,004XX N MAY ST,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,OTHER (SPECIFY),False,False,1214,12.00,27.00,24.00,26,NaN,NaN,2023,04/01/2023 04:49:15 PM,NaN,NaN,NaN


In [25]:
(chicago_full.isna().sum()/len(chicago_full)).round(2)

ID                     0.00
Case Number            0.00
Date                   0.00
Block                  0.00
IUCR                   0.00
Primary Type           0.00
Description            0.00
Location Description   0.00
Arrest                 0.00
Domestic               0.00
Beat                   0.00
District               0.00
Ward                   0.08
Community Area         0.08
FBI Code               0.00
X Coordinate           0.01
Y Coordinate           0.01
Year                   0.00
Updated On             0.00
Latitude               0.01
Longitude              0.01
Location               0.01
dtype: float64

In [26]:
## Dropping unneeded columns
drop_cols = ["X Coordinate","Y Coordinate", "Community Area","FBI Code",
             "Case Number","Updated On",'Block','Location','IUCR','Date']

In [28]:
# save final df
chicago_final = chicago_full.drop(columns=drop_cols).sort_index()#.reset_index()
chicago_final.head()

,ID,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Year,Latitude,Longitude
Datetime,,,,,,,,,,,,
2001-01-01 01:00:00,6154338,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,831,8.00,15.00,2001,41.77,-87.70
2001-01-01 01:00:00,1311503,CRIM SEXUAL ASSAULT,AGGRAVATED: OTHER DANG WEAPON,BAR OR TAVERN,False,False,1933,19.00,NaN,2001,41.93,-87.65
2001-01-01 01:00:00,1318099,BATTERY,SIMPLE,RESIDENCE PORCH/HALLWAY,False,True,214,2.00,NaN,2001,41.82,-87.62
2001-01-01 01:00:00,1311358,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,914,9.00,NaN,2001,41.81,-87.69
2001-01-01 01:00:00,1422085,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,True,1023,10.00,NaN,2001,41.86,-87.70


In [29]:
chicago_final.tail()

,ID,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Year,Latitude,Longitude
Datetime,,,,,,,,,,,,
2023-03-25 12:50:00,13021773,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,1723,17.00,45.00,2023,41.95,-87.72
2023-03-25 12:50:00,13021160,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,2515,25.00,36.00,2023,41.92,-87.77
2023-03-25 12:50:00,13021465,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,OTHER (SPECIFY),False,False,1214,12.00,27.00,2023,NaN,NaN
2023-03-25 12:55:00,13020692,ROBBERY,ARMED - HANDGUN,SIDEWALK,False,False,1935,19.00,43.00,2023,41.93,-87.64
2023-03-25 12:59:00,13021224,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,STREET,True,False,132,1.00,3.00,2023,41.86,-87.62


### Create two forms of data

1. Original format with each row as one crime (`Datetime` index)
2. A resampled/converted crime counts version (Each row is 1 day)

In [30]:
# form 1
df_crime = chicago_final.copy()

In [32]:
# create form 2
# calculate month, hour based on date
df_byday = df_crime.copy()
df_byday = df_byday.reset_index()
df_byday['Month'] = df_byday['Datetime'].dt.month_name()
df_byday['Hour'] = df_byday['Datetime'].dt.hour
df_byday

,Datetime,ID,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Year,Latitude,Longitude,Month,Hour
0,2001-01-01 01:00:00,6154338,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,831,8.00,15.00,2001,41.77,-87.70,January,1
1,2001-01-01 01:00:00,1311503,CRIM SEXUAL ASSAULT,AGGRAVATED: OTHER DANG WEAPON,BAR OR TAVERN,False,False,1933,19.00,NaN,2001,41.93,-87.65,January,1
2,2001-01-01 01:00:00,1318099,BATTERY,SIMPLE,RESIDENCE PORCH/HALLWAY,False,True,214,2.00,NaN,2001,41.82,-87.62,January,1
3,2001-01-01 01:00:00,1311358,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,914,9.00,NaN,2001,41.81,-87.69,January,1
4,2001-01-01 01:00:00,1422085,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,True,1023,10.00,NaN,2001,41.86,-87.70,January,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7765324,2023-03-25 12:50:00,13021773,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,1723,17.00,45.00,2023,41.95,-87.72,March,12
7765325,2023-03-25 12:50:00,13021160,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,2515,25.00,36.00,2023,41.92,-87.77,March,12
7765326,2023-03-25 12:50:00,13021465,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,OTHER (SPECIFY),False,False,1214,12.00,27.00,2023,NaN,NaN,March,12
7765327,2023-03-25 12:55:00,13020692,ROBBERY,ARMED - HANDGUN,SIDEWALK,False,False,1935,19.00,43.00,2023,41.93,-87.64,March,12


In [34]:
df_byday['Hour'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])